# Iteration 2: Summary

I kept the same learning rate as the last iteration, but changed the batch size to 64 and ran for 40 epochs.

Test results:

Accuracy 0.2987
Loss 1.6089

In [8]:
'''
import shutil
import os

source = './food-101/test/spaghetti_carbonara'
dest = './food-101/valid/spaghetti_carbonara'

files = os.listdir(source)

i = 0
for f in files:
    if i < 175:
        file= os.path.join(source, f)
        shutil.move(file,dest)
        print("adding img ", i)
    i+=1
'''

'\nimport shutil\nimport os\n\nsource = \'./food-101/test/spaghetti_carbonara\'\ndest = \'./food-101/valid/spaghetti_carbonara\'\n\nfiles = os.listdir(source)\n\ni = 0\nfor f in files:\n    if i < 175:\n        file= os.path.join(source, f)\n        shutil.move(file,dest)\n        print("adding img ", i)\n    i+=1\n'

In [1]:
# 1 RUN ME
import numpy as np
import matplotlib.pyplot as plt

import os
from os import listdir
from os.path import isfile, join

import cv2
import random
import pickle
import datetime
import time
import shutil
import stat

import collections
from collections import defaultdict

import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.utils import to_categorical, plot_model

from keras.models import Model

Using TensorFlow backend.


In [2]:
# 2 RUN ME
# constants
IMG_SIZE = 224
NUM_CLASSES = 5
noodle_classes = ['pad_thai','pho','ramen','spaghetti_bolognese','spaghetti_carbonara']

EPOCHS = 40
BATCH_SZ = 64
LEARN_RATE = 0.0001

In [3]:
# 3 RUN ME
def prepare(filepath):
    print('fp:',filepath)
    try:
        #img = os.listdir(filepath)
        #print(img)
        img_arr = cv2.imread(filepath) # each image is a 2-d array of RGB values
        img_to_rgb = img_arr[:,:,::-1] # convert image to rgb
        new_img = cv2.resize(img_to_rgb, (IMG_SIZE,IMG_SIZE)) # indexed to remove dtype=uint8
        #plt.imshow(new_img)
        #plt.show()
    except Exception as e:
        print("create exception: ",e)
    
    new_img = np.array(new_img).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
    new_img = tf.cast(new_img,tf.float32)
    #new_img = new_img.astype('float32')
    #print(new_img.dtype)
    #new_img = tf.convert_to_tensor(new_img,dtype=tf.float32)
    return new_img

In [4]:
# 4 RUN ME
root_dir = 'food-101/images/'

# this function pre-processes every food image
# and assigns label to the respective food class
# shoves everything into an array called data_set
# shuffles the array
# then separates the data_set array into features array and label array
def create_data(root, _name):
    data_set = []

    # pre-process: re-size and re-color
    for noodle_class in noodle_classes:
        path = os.path.join(root, noodle_class)
        label = noodle_classes.index(noodle_class) # FIX LATER: use enumerate
        print(noodle_class,label)
        #input('wait1')

        for img in os.listdir(path): # pad_thai/1432432.jpg, pad_thai/12314.jpg . . .
                if img.startswith('.'): # .DS_Store
                    continue
                # each image is a 2D array of RGB value
                try:
                    img_arr = cv2.imread(os.path.join(path,img))
                    img_to_rgb = img_arr[:,:,::-1] 
                    new_img = cv2.resize(img_to_rgb,(IMG_SIZE,IMG_SIZE)) 
                    data_set.append([new_img,label]) # shove (X,y) in here cause we wanna shuffle 
                except Exception as e:
                    print("create exception: ",e)
        
    # randomize
    random.shuffle(data_set)
    random.shuffle(data_set)
    
    X = []
    y = []

    # now separate X and y 
    # X (features) is a 2D array of RGB values
    # y (labels) is a 1D array of integers
    for features, label in data_set:
        X.append(features)
        y.append(label)
        #print('feature loop')
        #print(features, label)
        #input('wait')

    # re-shape for training
    X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3) # -1 = any number of features (catch-all, it'll recognize)
    y = np.array(y).reshape(-1, 1)
    print('X', X.shape)
    print('y', y.shape)
    
    #save(X,y,_name)

    return X,y

In [5]:
# 5 RUN ME
# load data

X_train, y_train = create_data('food-101/train', "train")
X_test, y_test = create_data('food-101/test', "test")
X_valid, y_valid = create_data('food-101/valid', "valid")

pad_thai 0
pho 1
ramen 2
spaghetti_bolognese 3
spaghetti_carbonara 4
X (3750, 224, 224, 3)
y (3750, 1)
pad_thai 0
pho 1
ramen 2
spaghetti_bolognese 3
spaghetti_carbonara 4
X (375, 224, 224, 3)
y (375, 1)
pad_thai 0
pho 1
ramen 2
spaghetti_bolognese 3
spaghetti_carbonara 4
X (875, 224, 224, 3)
y (875, 1)


In [6]:
%load_ext tensorboard

In [17]:
%tensorboard --logdir logs

In [6]:
# 6 RUN ME
# normalize (before feeding data into network)
# X_train = X_train/255.0
# X_test = X_test/255.0

#print('X train before normalize',X_train[1])
print('y train before normal',y_train)
# input('wait1')
X_train = tf.keras.utils.normalize(X_train, axis=1)
y_train = tf.keras.utils.to_categorical(y_train, NUM_CLASSES)

#print('X train after normal',X_train[1])
print('y train after normal',y_train)
# input('wait2')

X_test = tf.keras.utils.normalize(X_test, axis=1)
y_test = tf.keras.utils.to_categorical(y_test, NUM_CLASSES)

#print('y test',y_test)

X_valid = tf.keras.utils.normalize(X_valid, axis=1)
y_valid = tf.keras.utils.to_categorical(y_valid, NUM_CLASSES)b

y train before normal [[4]
 [3]
 [0]
 ...
 [2]
 [0]
 [0]]
y train after normal [[0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]
 ...
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]]


# VGGNet CNN (Working)

In [7]:
# 7 RUN ME - this is the one that takes hours
## WORKING!!!!
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
input_shape = (IMG_SIZE, IMG_SIZE, 3)

#Instantiate an empty model
model = Sequential([
Conv2D(64, (3, 3), input_shape=input_shape, padding='same', activation='relu'),
Conv2D(64, (3, 3), activation='relu', padding='same'),
MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
Conv2D(128, (3, 3), activation='relu', padding='same'),
Conv2D(128, (3, 3), activation='relu', padding='same',),
MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
Conv2D(256, (3, 3), activation='relu', padding='same',),
Conv2D(256, (3, 3), activation='relu', padding='same',),
Conv2D(256, (3, 3), activation='relu', padding='same',),
MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
Conv2D(512, (3, 3), activation='relu', padding='same',),
Conv2D(512, (3, 3), activation='relu', padding='same',),
Conv2D(512, (3, 3), activation='relu', padding='same',),
MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
Conv2D(512, (3, 3), activation='relu', padding='same',),
Conv2D(512, (3, 3), activation='relu', padding='same',),
Conv2D(512, (3, 3), activation='relu', padding='same',),
MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
Flatten(),
Dense(4096, activation='relu'),
Dense(4096, activation='relu'),
Dense(5, activation='softmax')
])

model.summary()

opt = optimizers.SGD(lr=LEARN_RATE, momentum=.9)

model.compile(loss="categorical_crossentropy",
              optimizer=opt,
              metrics=['accuracy'])
'''


# Compile the model
model.compile(loss="categorical_crossentropy", optimizer='SGD', metrics=['accuracy'])
'''
print(X_train.shape)
print(y_train.shape)

model.fit(X_train, 
          y_train, 
          batch_size=BATCH_SZ, 
          epochs=EPOCHS,     
          validation_data=(X_valid, y_valid))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 56, 56, 256)      

In [8]:
# 8 RUN ME
# evaluate model on test data
scores = model.evaluate(X_test, y_test, verbose=1)
# verbose indiciator to display training progress info
# 1 (true) (default) | 0 (false)

print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

375/375 [==============================] - 23s 61ms/step
Test loss: 1.608901547431946
Test accuracy: 0.29866665601730347


# Ignore below (not working)

In [22]:

# NOT WORKING
# Example of creating a CNN model with many VGG blocks
from keras.models import Model
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.utils import plot_model

# function for creating a vgg block
def vgg_block(layer_in, n_filters, n_conv):
    # add convolutional layers
    for _ in range(n_conv):
        layer_in = Conv2D(n_filters, (3,3), padding='same', activation='relu')(layer_in)
    # add max pooling layer
    layer_in = MaxPooling2D((2,2), strides=(2,2))(layer_in)
    print(layer_in.shape)
    return layer_in

# define model input
visible = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
# add vgg module
layer = vgg_block(visible, 64, 2)
# add vgg module
layer = vgg_block(layer, 128, 2)
# add vgg module
layer = vgg_block(layer, 256, 4)

# create model
model = Model(inputs=visible, outputs=layer)
# summarize model
model.summary()
# plot model architecture
plot_model(model, show_shapes=True, to_file='multiple_vgg_blocks.png')

opt = optimizers.SGD(lr=0.01, momentum=.9)

model.compile(loss="categorical_crossentropy",
              optimizer=opt,
              metrics=['accuracy'])

model.fit(X_train, 
          y_train, 
          batch_size=32, # batch_size is how much data a time we wanna pass through a layer
          epochs=20,      # epochs = how many times go through the network
          validation_split=(X_valid, y_valid))


(None, 112, 112, 64)
(None, 56, 56, 128)
(None, 28, 28, 256)
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 112, 112, 128)     147584    
________________________________________________________________

ValueError: Error when checking target: expected max_pooling2d_8 to have 4 dimensions, but got array with shape (3750, 5)

In [15]:
'''
# NOT WORKING
# from github
import tensorflow as tf
import tensorflow.keras as keras

from keras.models import Model
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.utils import plot_model

'''
# function for creating a vgg block
def vgg_block(layer_in, n_filters, n_conv):
    # add convolutional layers
    for _ in range(n_conv):
        layer_in = Conv2D(n_filters, (3,3), padding='same', activation='relu')(layer_in)
    # add max pooling layer
    layer_in = MaxPooling2D((2,2), strides=(2,2))(layer_in)
    print(layer_in.shape)
    return layer_in
    
# specify model
model = Sequential()

# layer 1: input shape mandatory
model.add(Conv2D(conv_filters_1, (3,3), strides=(1,1), padding='same',input_shape = X_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
'''
model = keras.Sequential()

# Block 1
img_input = Input(shape=(IMG_SIZE, IMG_SIZE, 3))
print("image input shape: ", img_input.shape)
x = Conv2D(64, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block1_conv1')(img_input)
print("layer 1, block1_conv1 shape: ", x.shape)
x = Conv2D(64, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block1_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

# Block 2
x = Conv2D(128, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block2_conv1')(x)
x = Conv2D(128, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block2_conv2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

# Block 3
x = Conv2D(256, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block3_conv1')(x)
x = Conv2D(256, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block3_conv2')(x)
x = Conv2D(256, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block3_conv3')(x)
x = Conv2D(256, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block3_conv4')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

# Block 4
x = Conv2D(512, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block4_conv1')(x)
x = Conv2D(512, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block4_conv2')(x)
x = Conv2D(512, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block4_conv3')(x)
x = Conv2D(512, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block4_conv4')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

# Block 5
x = Conv2D(512, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block5_conv1')(x)
x = Conv2D(512, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block5_conv2')(x)
x = Conv2D(512, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block5_conv3')(x)
x = Conv2D(512, (3, 3),
                  activation='relu',
                  padding='same',
                  name='block5_conv4')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)
print("last layer, block5_pool shape: ", x.shape)
#x = keras.layers.GlobalMaxPooling2D()(x)

# Create model.
model = Model(img_input, x, name='vgg19')

#model = Model(inputs=img_input, outputs=x)
model.summary()


# plot model architecture
plot_model(model, show_shapes=True, to_file='multiple_vgg_blocks.png')

opt = optimizers.SGD(lr=0.01, momentum=.9)

model.compile(loss="categorical_crossentropy",
              optimizer=opt,
              metrics=['accuracy'])
print("X_train shape: ", X_train.shape)
print("y_train shape: ", y_train.shape)
model.fit(X_train, 
          y_train, 
          batch_size=32, # batch_size is how much data a time we wanna pass through a layer
          epochs=20,      # epochs = how many times go through the network
          validation_split=(X_valid, y_valid))
'''

image input shape:  (None, 128, 128, 3)
layer 1, block1_conv1 shape:  (None, 128, 128, 64)
last layer, block5_pool shape:  (None, 4, 4, 512)
Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)    

ValueError: Error when checking target: expected block5_pool to have 4 dimensions, but got array with shape (3750, 5)

In [ ]:
# evaluate model on test data
scores = model.evaluate(X_test, y_test, verbose=1)
# verbose indiciator to display training progress info
# 1 (true) (default) | 0 (false)
print(NAME)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [ ]:
# classify a random image
#X = prepare('/Users/flo/Desktop/456/test/carb.jpg')
# prediction = model.predict([prepare('/Users/flo/Desktop/456/test/carb.jpg')]) 
# this prediction should give real values (percentages) for each class like 70% sure this class is this

print(prediction.shape)
prediction = model.predict_classes(X_test[:10])

print('Predicted class: ')
print(prediction)

print('Real class: ')
print(y_test[:10])

model_accuracies = model.predict(X_test[:10])
print('\nAccuracy of each model')
print(model_accuracies[:10])

print('\n(pretty)Accuracy of each model')
print(np.round(model_accuracies[:10],3))
# prediction = model.predict([prepare('/Users/flo/Desktop/456/test/carb.jpg')])
# print("carb:", prediction)
# print(noodle_classes[int(prediction[0][0])])


# prediction = model.predict([prepare('/Users/flo/Desktop/456/test/padthai.jpg')])
# print("pdt:", prediction)
# print(noodle_classes[int(prediction[0][0])])


# prediction = model.predict([prepare('/Users/flo/Desktop/456/test/pho.jpg')])
# print("ph:", prediction)
# print(noodle_classes[int(prediction[0][0])])


# prediction = model.predict([prepare('/Users/flo/Desktop/456/test/ramen.jpg')])
# print("ram:", prediction)
# print(noodle_classes[int(prediction[0][0])])


In [ ]:

#['spaghetti_bolognese', 'pad_thai', 'spaghetti_carbonara', 'ramen', 'pho']

prediction = model.predict([prepare('/Users/flo/Desktop/456/test/carb.jpg')])
#prediction2 = model.predict([prepare('/Users/flo/Desktop/sc.jpg')])

print(prediction)
#print(prediction2)
#input('wait')
#print(CATEGORIES[int(prediction[0][0])])
#print(CATEGORIES[int(prediction2[0][0])])

In [ ]:
#Right before the last layer. This is generally a bad place to apply dropout, 
#because the network has no ability to "correct" errors induced by dropout 
#before the classification happens. If I read correctly, you might have put 
#dropout right before the softmax in the iris MLP.

#our data set provided us with the training data and test data [1:3] ratio